### Imagination continued

This notebook contains code for getting samples from a given class with Monte Carlo sampling

Tested with tensorflow 2.11.0 and Python 3.10.9.

In [ ]:
import json
import zipfile
import os
import psutil
import numpy as np
import pandas as pd
import numpy as np
from tensorflow import keras
from PIL import Image, ImageOps
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from tensorflow.keras import Model, Sequential, metrics, optimizers, layers
from tensorflow.python.framework.ops import disable_eager_execution
from utils import load_tfds_dataset
from generative_model import encoder_network_large, decoder_network_large, VAE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

tf.keras.utils.set_random_seed(123)

#### Load model

In [ ]:
K.set_image_data_format('channels_last')

latent_dim = 20
input_shape = (64, 64, 3)

encoder, z_mean, z_log_var = encoder_network_large(input_shape, latent_dim)
decoder = decoder_network_large(latent_dim)

encoder.load_weights("model_weights/shapes3d_encoder.h5")
decoder.load_weights("model_weights/shapes3d_decoder.h5")

#### Train classifier

(Identical method to classifier used to measure decoding accuracy over time.)

In [ ]:
train_ds, test_ds, train_labels, test_labels = load_tfds_dataset('shapes3d', labels=True, 
                                                                 key_dict= {'shapes3d': 'label_shape'})
train_ds = train_ds / 255
test_ds = test_ds / 255

In [ ]:
latents = encoder.predict(test_ds)
clf = make_pipeline(StandardScaler(), SVC(probability=True))
clf.fit(latents[0], test_labels)

#### Sampling

In [ ]:
# perform Monte Carlo sampling to generate new samples
n_samples = 10000  # number of samples to generate
samples = np.random.normal(loc=0, scale=1, size=(n_samples, latent_dim))

# predict the labels of the new samples
prob = clf.predict_proba(samples) 

class_idx = 0

# get the probabilities for the desired class
class_probabilities = prob[:, class_idx]
# sort the samples by the probabilities, in descending order 
sorted_samples = samples[np.argsort(class_probabilities)[::-1]]
# decode the high-scoring samples into images
decoded_images = decoder.predict(sorted_samples)

n = 20 # number of images to display
plt.figure(figsize=(40, 4))

for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(decoded_images[i].reshape(64, 64, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()